In [12]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import preprocess_data 

reload(preprocess_data)

%matplotlib inline

In [13]:
measurements_blood_pressure = pd.read_csv('/home/bhatti/dataset/VCHAMPS/measurements_blood_pressure_train.csv').iloc[:,1:]

In [14]:
measurements_blood_pressure

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State
0,1,61.554040,2005-10-25 00:02:08.0,75.0,140.0,Indiana
1,1,67.037260,2011-04-21 02:50:27.0,72.0,116.0,Indiana
2,1,68.304140,2012-07-27 03:54:47.0,100.0,145.0,Indiana
3,1,68.347339,2012-08-11 22:51:23.0,89.0,155.0,Indiana
4,1,68.781623,2013-01-17 16:23:39.0,72.0,143.0,Indiana
...,...,...,...,...,...,...
21997553,99999,96.324828,2013-03-28 17:23:42.0,62.0,147.0,Arizona
21997554,99999,96.326070,2013-03-29 04:17:14.0,57.0,123.0,Arizona
21997555,99999,96.331619,2013-03-31 04:57:36.0,68.0,154.0,Arizona
21997556,99999,96.357136,2013-04-09 12:47:57.0,79.0,147.0,Arizona


In [15]:
path = '/home/hassan/lily/MLA/FDA/inpatient_full_simple.csv'
inpatient = pd.read_csv(path).iloc[:,1:]
inpatient.head()

,Internalpatientid,num_stays,stay_length,num_unique_units,num_transfers,num_cvd_admission,Died,AO,CVD,unique_admitting_specialty,...,freq,Medical,Mental,Others_Specialty,Rehab,Gerontology,CVD_readmission,next_readmission_time,Discharge date,readmission within 300 days
0,1,3,14.16,2,0,0,0,0,0,3,...,0.6,3,0,0,0,0,0,2352.34,2016-07-22 21:36:24.0,0
1,2,21,71.17,5,2,10,0,0,1,9,...,1.5,10,1,0,9,1,1,0.00,2023-09-12 02:28:10.0,0
2,3,1,1.83,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,885.57,2002-03-30 18:12:14.0,0
3,4,1,7.15,1,0,1,0,0,1,1,...,1.0,0,0,0,1,0,0,0.00,2015-02-23 09:46:15.0,0
4,5,1,1.04,1,0,1,0,0,1,1,...,1.0,1,0,0,0,0,0,26.75,2004-10-02 06:54:56.0,1


In [16]:
inpatient_date = inpatient[['Internalpatientid','Discharge date']]
inpatient_date

,Internalpatientid,Discharge date
0,1,2016-07-22 21:36:24.0
1,2,2023-09-12 02:28:10.0
2,3,2002-03-30 18:12:14.0
3,4,2015-02-23 09:46:15.0
4,5,2004-10-02 06:54:56.0
...,...,...
84531,169055,2002-04-12 16:51:53.0
84532,169057,2025-02-12 17:23:04.0
84533,169060,2017-02-08 09:55:03.0
84534,169062,2005-06-04 03:34:07.0


In [17]:
# Filter procedure date that's before discharge date
measurements_bp_merged = measurements_blood_pressure.merge(inpatient_date, how = 'left', on = 'Internalpatientid')
measurements_bp_merged = measurements_bp_merged.loc[measurements_bp_merged['Measurement date'] <= measurements_bp_merged['Discharge date']]
measurements_bp_merged 

,Internalpatientid,Age at measurement bp,Measurement date,Diastolic bp,Systolic bp,State,Discharge date
0,1,61.554040,2005-10-25 00:02:08.0,75.0,140.0,Indiana,2016-07-22 21:36:24.0
1,1,67.037260,2011-04-21 02:50:27.0,72.0,116.0,Indiana,2016-07-22 21:36:24.0
2,1,68.304140,2012-07-27 03:54:47.0,100.0,145.0,Indiana,2016-07-22 21:36:24.0
3,1,68.347339,2012-08-11 22:51:23.0,89.0,155.0,Indiana,2016-07-22 21:36:24.0
4,1,68.781623,2013-01-17 16:23:39.0,72.0,143.0,Indiana,2016-07-22 21:36:24.0
...,...,...,...,...,...,...,...
21997553,99999,96.324828,2013-03-28 17:23:42.0,62.0,147.0,Arizona,2013-04-14 10:02:01.0
21997554,99999,96.326070,2013-03-29 04:17:14.0,57.0,123.0,Arizona,2013-04-14 10:02:01.0
21997555,99999,96.331619,2013-03-31 04:57:36.0,68.0,154.0,Arizona,2013-04-14 10:02:01.0
21997556,99999,96.357136,2013-04-09 12:47:57.0,79.0,147.0,Arizona,2013-04-14 10:02:01.0


In [18]:
pd.isnull(measurements_bp_merged).sum()

Internalpatientid        0
Age at measurement bp    0
Measurement date         0
Diastolic bp             0
Systolic bp              0
State                    0
Discharge date           0
dtype: int64

In [19]:
bins = [0, 60, 80, 200]
diastolic_bins = pd.crosstab(measurements_bp_merged['Internalpatientid'], pd.cut(measurements_bp_merged['Diastolic bp'], bins)).reset_index()

In [20]:
internalpatientid = pd.read_csv('/home/vivi/FDA_datasets/outpatient_state.csv').iloc[:,1:]['Internalpatientid'].reset_index()

In [21]:
internalpatientid.drop(columns='index',inplace=True)

In [22]:
internalpatientid = measurements_bp_merged.groupby('Internalpatientid')['Internalpatientid']


In [23]:
internalpatientid 

In [24]:
# hypotension
bins = [20,40,60,80,100, 120]
previous_df = inpatient_date
data = measurements_bp_merged['Internalpatientid'].loc[(measurements_bp_merged['Systolic bp'] < 100) | (measurements_bp_merged['Diastolic bp'] < 60)]
new_df = pd.crosstab(data, pd.cut(measurements_bp_merged['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypotension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypotension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypotension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypotension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypotension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [25]:
# healthy
data = measurements_bp_merged['Internalpatientid'].loc[(measurements_bp_merged['Systolic bp'] >= 100) & (measurements_bp_merged['Systolic bp'] < 130) 
                                                            & (measurements_bp_merged['Diastolic bp'] >= 60) & (measurements_bp_merged['Diastolic bp'] < 80)]
new_df = pd.crosstab(data, pd.cut(measurements_bp_merged['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 healthy',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 healthy',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 healthy',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 healthy',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 healthy'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [26]:
# hypertension
data = measurements_bp_merged['Internalpatientid'].loc[(measurements_bp_merged['Systolic bp'] > 130) | (measurements_bp_merged['Diastolic bp'] > 80) ]
new_df = pd.crosstab(data, pd.cut(measurements_bp_merged['Age at measurement bp'], bins)).reset_index().fillna(0)
new_df = new_df.rename(columns={pd.Interval(20, 40, closed='right'): 'Age 20-40 hypertension',
                                pd.Interval(40, 60, closed='right'): 'Age 40-60 hypertension',
                                pd.Interval(60, 80, closed='right'): 'Age 60-80 hypertension',
                                pd.Interval(80, 100, closed='right'): 'Age 80-100 hypertension',
                                pd.Interval(100, 120, closed='right'): 'Age 100-120 hypertension'})
previous_df = previous_df.merge(new_df, how = 'left', on = 'Internalpatientid')

In [27]:
previous_df

,Internalpatientid,Discharge date,Age 20-40 hypotension,Age 40-60 hypotension,Age 60-80 hypotension,Age 80-100 hypotension,Age 100-120 hypotension,Age 20-40 healthy,Age 40-60 healthy,Age 60-80 healthy,Age 80-100 healthy,Age 100-120 healthy,Age 20-40 hypertension,Age 40-60 hypertension,Age 60-80 hypertension,Age 80-100 hypertension,Age 100-120 hypertension
0,1,2016-07-22 21:36:24.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,34.0,0.0,0.0,0.0,4.0,134.0,0.0,0.0
1,2,2023-09-12 02:28:10.0,0.0,6.0,86.0,0.0,0.0,0.0,23.0,231.0,0.0,0.0,0.0,52.0,212.0,0.0,0.0
2,3,2002-03-30 18:12:14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-23 09:46:15.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,3.0,14.0,0.0,0.0,0.0,3.0,38.0,0.0
4,5,2004-10-02 06:54:56.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84531,169055,2002-04-12 16:51:53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,0.0,0.0,0.0
84532,169057,2025-02-12 17:23:04.0,0.0,0.0,37.0,156.0,0.0,0.0,0.0,109.0,221.0,0.0,0.0,0.0,247.0,217.0,0.0
84533,169060,2017-02-08 09:55:03.0,0.0,2.0,30.0,0.0,0.0,0.0,4.0,57.0,0.0,0.0,0.0,43.0,42.0,0.0,0.0
84534,169062,2005-06-04 03:34:07.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,41.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0


In [28]:
pd.crosstab(measurements_bp_merged['Internalpatientid'].loc[(measurements_bp_merged['Age at measurement bp'] > 20) & (measurements_bp_merged['Age at measurement bp'] <= 40)], pd.cut(measurements_bp_merged['Diastolic bp'], bins)).reset_index()

Diastolic bp,Internalpatientid,"(20, 40]","(40, 60]","(60, 80]","(80, 100]","(100, 120]"
0,201,0,3,72,139,13
1,265,0,2,21,0,0
2,275,0,0,11,3,0
3,402,0,0,3,11,3
4,406,0,0,1,1,1
...,...,...,...,...,...,...
2152,168690,0,0,1,19,20
2153,168704,0,0,1,1,0
2154,168844,0,4,12,9,0
2155,168926,0,0,0,3,0


In [29]:
measurements_bp_preprocessed = previous_df

In [30]:
measurements_bp_preprocessed.to_csv('/home/vivi/FDA_datasets/inpatient_measurements_bp_preprocessed.csv')